In [1]:
import numpy as np
from simpy_helpers import Entity, Resource, Source, Stats
import simpy

In [2]:
NUM_CUSTOMERS = 100
MEAN_TIME_BETWEEN_ARRIVALS = 100
MEAN_COOK_TIME_DURATION = 390
MEAN_EATING_DURATION = 5000
NUM_COOKS = 4
NUM_SEATS = 5

In [3]:
# Here is the Resource subclass
class Cooks(Resource):
    def service_time(self, entity):
        return np.random.exponential(MEAN_COOK_TIME_DURATION)
    
class Seats(Resource):
    def service_time(self, entity):
        return np.random.exponential(MEAN_EATING_DURATION)

In [4]:
## Here is the Entity Subclass
class Job(Entity):
    def process(self):
        yield self.wait_for_resource(cook)
        yield self.process_at_resource(cook)
        self.release_resource(cook)
        
        yield self.wait_for_resource(seat)
        yield self.process_at_resource(seat)
        self.release_resource(seat)

In [5]:
# Here is the Source subclass
class GenerateCustomers(Source):
    def interarrival_time(self):
#         return np.random.exponential(MEAN_TIME_BETWEEN_ARRIVALS)
        return MEAN_TIME_BETWEEN_ARRIVALS
    
    def build_entity(self):
        return Job(env) 

In [6]:
env = simpy.Environment()

seat = Seats(env, capacity=NUM_SEATS) 
cook = Cooks(env, capacity=NUM_COOKS) 
source = GenerateCustomers(env, number=NUM_CUSTOMERS) 


env.process(source.start())
env.run()

In [7]:
# loop through all the jobs
system_times = Stats.get_total_times()

print(f'average time in system is {np.mean(system_times)}')

average time in system is 59420.13349844848


In [8]:
wait_times = Stats.get_waiting_times()
print("avg waiting time:", np.sum(wait_times)/NUM_CUSTOMERS) 
processing_times = Stats.get_processing_times()
print("avg processing time:", np.sum(processing_times)/NUM_CUSTOMERS) 


avg waiting time: 52977.05514387454
avg processing time: 6443.078354573939


In [9]:
cook_wait_times = Stats.get_waiting_times(cook)
print("avg waiting time for a cook resource", np.sum(cook_wait_times)/NUM_CUSTOMERS)
seat_wait_times = Stats.get_waiting_times(seat)
print("avg waiting time for a seat resource", np.sum(seat_wait_times)/NUM_CUSTOMERS)

avg waiting time for a cook resource 161.4298198049126
avg waiting time for a seat resource 52815.62532406963


In [10]:
cook_processing_times = Stats.get_processing_times(cook)
print("avg processing time for a cook resource", np.sum(cook_processing_times)/NUM_CUSTOMERS)
seat_processing_times = Stats.get_processing_times(seat)
print("avg processing time for a seat resource", np.sum(seat_processing_times)/NUM_CUSTOMERS)

avg processing time for a cook resource 372.7626706104078
avg processing time for a seat resource 6070.315683963531


In [11]:
queue_size_for_cooks = Stats.queue_size_over_time(cook)
# print(queue_size_for_cooks)
print("avg queue size for cooks", np.sum(queue_size_for_cooks)/len(queue_size_for_cooks))


avg queue size for cooks 0.12592708130239813


In [12]:
queue_size_for_seats = Stats.queue_size_over_time(seat)
# print(queue_size_for_seats)
print("avg queue size for seats", np.sum(queue_size_for_seats)/len(queue_size_for_seats))

avg queue size for seats 41.189822577500486


In [13]:
cooks_utilization = Stats.utilization_over_time(cook)
print("cooks utilization", np.sum(cooks_utilization)/len(cooks_utilization))

cooks utilization 0.07267693507506337


In [14]:
seats_utilization = Stats.utilization_over_time(seat)
print("seats utilization", np.sum(seats_utilization)/len(seats_utilization))

seats utilization 0.9468216026515891
